# Age

In [1]:
import jupyter_black
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split

In [2]:
jupyter_black.load()

# Prepare

In [3]:
df = pd.read_csv("./datasets/prepared.csv").set_index("PassengerId")
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,isTest,Name_FirstName,Name_Title,Name_LastName,FamilySize,isAlone
PassengerId,,,,,,,,,,,,,,,,
1,0.0,3,1,22.0,1,0,0,-0.503595,NaN,2,0,29,6,150,2,0
2,1.0,1,0,38.0,1,0,12,0.734503,C85,0,0,61,7,104,2,0
3,1.0,3,0,26.0,0,0,21,-0.490544,NaN,2,0,175,4,149,1,1
4,1.0,1,0,35.0,1,0,26,0.382925,C123,2,0,88,7,96,2,0
5,0.0,3,1,35.0,0,0,26,-0.488127,NaN,2,0,5,6,182,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,NaN,3,1,NaN,0,0,2,-0.488127,NaN,2,1,175,6,149,1,1
1306,NaN,1,0,39.0,0,0,12,1.461829,C105,0,1,175,9,149,1,1
1307,NaN,3,1,38.5,0,0,19,-0.503595,NaN,2,1,175,6,149,1,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 1 to 1309
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Survived        891 non-null    float64
 1   Pclass          1309 non-null   int64  
 2   Sex             1309 non-null   int64  
 3   Age             1046 non-null   float64
 4   SibSp           1309 non-null   int64  
 5   Parch           1309 non-null   int64  
 6   Ticket          1309 non-null   int64  
 7   Fare            1309 non-null   float64
 8   Cabin           295 non-null    object 
 9   Embarked        1309 non-null   int64  
 10  isTest          1309 non-null   int64  
 11  Name_FirstName  1309 non-null   int64  
 12  Name_Title      1309 non-null   int64  
 13  Name_LastName   1309 non-null   int64  
 14  FamilySize      1309 non-null   int64  
 15  isAlone         1309 non-null   int64  
dtypes: float64(3), int64(12), object(1)
memory usage: 173.9+ KB


In [5]:
X = df.drop(["Survived", "Age", "Cabin", "isTest"], axis=1)
X

,Pclass,Sex,SibSp,Parch,Ticket,Fare,Embarked,Name_FirstName,Name_Title,Name_LastName,FamilySize,isAlone
PassengerId,,,,,,,,,,,,
1,3,1,1,0,0,-0.503595,2,29,6,150,2,0
2,1,0,1,0,12,0.734503,0,61,7,104,2,0
3,3,0,0,0,21,-0.490544,2,175,4,149,1,1
4,1,0,1,0,26,0.382925,2,88,7,96,2,0
5,3,1,0,0,26,-0.488127,2,5,6,182,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,1,0,0,2,-0.488127,2,175,6,149,1,1
1306,1,0,0,0,12,1.461829,0,175,9,149,1,1
1307,3,1,0,0,19,-0.503595,2,175,6,149,1,1


In [6]:
y = df[["Age"]]
y

,Age
PassengerId,
1,22.0
2,38.0
3,26.0
4,35.0
5,35.0
...,...
1305,NaN
1306,39.0
1307,38.5


In [7]:
test_ids = y[y["Age"].isna()].index

X_train = X[~X.index.isin(test_ids)]
X_test = X[X.index.isin(test_ids)]

y_train = y[~y.index.isin(test_ids)]
y_test = y[y.index.isin(test_ids)]

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1046, 12), (263, 12), (1046, 1), (263, 1))

In [8]:
X_train, X_true, y_train, y_true = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42
)
X_train.shape, X_true.shape, y_train.shape, y_true.shape

((941, 12), (105, 12), (941, 1), (105, 1))

# Train

In [9]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42
)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((846, 12), (95, 12), (846, 1), (95, 1))

In [10]:
model = CatBoostRegressor()

In [11]:
model.fit(Pool(X_train, y_train), verbose=False, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

# Predict

In [12]:
y_preds = model.predict(X_test)
y_preds

array([32.33351487, 31.26494687, 34.70994854, 26.51877661, 24.0597382 ,
       28.18379969, 57.43868711, 23.97622705, 28.29306124, 27.08594279,
       24.84250017, 27.30296372, 27.07755367, 23.33687272, 42.46771652,
       38.43322274,  3.02990754, 29.40599666, 28.17931677, 22.57463528,
       28.25814273, 28.4405472 , 24.24126495, 27.22283648, 23.17840256,
       28.72383725, 30.7989257 ,  7.97009524, 27.88666915, 31.17235093,
       26.50592764, 11.0763311 , 40.7599856 , 41.13328952,  5.93816195,
        9.88036828, 28.89850597, 46.14004742, 31.36529778, 32.35641045,
       24.10485751, 29.93875011, 24.17539074, 28.23461883,  5.25273938,
       23.44508222, 10.92902857, 18.27237758, 33.82431932, 43.9618699 ,
       40.60526185, 23.9074518 , 40.26802156, 24.08270091, 43.13786699,
       40.13624037, 39.8585056 , 43.23192303, 25.29602768, 26.32750772,
       27.22887744, 24.84080989, 32.06237396, 30.72403051, 14.73699488,
       42.70796671, 28.23461883, 29.83087784, 44.45612126, 26.51

In [13]:
y_test = pd.DataFrame({"PassengerId": X_test.index, "Age": y_preds}).set_index(
    "PassengerId"
)
y_test

,Age
PassengerId,
6,32.333515
18,31.264947
20,34.709949
27,26.518777
29,24.059738
...,...
1300,31.488843
1302,23.907452
1305,39.934307


In [14]:
df1 = df[["Age"]].copy()
df1

,Age
PassengerId,
1,22.0
2,38.0
3,26.0
4,35.0
5,35.0
...,...
1305,NaN
1306,39.0
1307,38.5


In [15]:
df1["isAgePreds"] = df1["Age"].isna().astype(int)

In [16]:
df1.fillna(y_test, inplace=True)

In [17]:
df1

,Age,isAgePreds
PassengerId,,
1,22.000000,0
2,38.000000,0
3,26.000000,0
4,35.000000,0
5,35.000000,0
...,...,...
1305,39.934307,1
1306,39.000000,0
1307,38.500000,0


# Save

In [18]:
df1.to_csv("./datasets/prepared_age.csv")